In [10]:
import pandas as pd
import sys
import os
import numpy as np

current_dir = os.path.abspath('')
parent_dir = os.path.join(current_dir, '..', 'functions')
sys.path.append(parent_dir)

from duplicates_utils import normalize_id
from mn_table_utils import create_mn_table

DROP DUPLICATES ROWS FROM AUTHORS AND CATEGORIES

In [ ]:
authors_path = "../../dataset/cleaned/authors_cleaned.csv"
df_authors = pd.read_csv(authors_path)

print(df_authors.shape)

df_authors,df_authors_map = normalize_id(df_authors, key_column = 'author_name', id_column = 'author_id')

print(df_authors.shape)
print(len(df_authors_map))

(654021, 2)
(653783, 2)
654021


In [4]:
categories_path = "../../dataset/cleaned/categories_cleaned.csv"
df_categories = pd.read_csv(categories_path)


print(df_categories.shape)

df_categories,df_categories_map = normalize_id(df_categories, key_column = 'category_name', id_column = 'category_id')

print(df_categories.shape)
print(len(df_categories_map))

(2775, 2)
(2585, 2)
2775


CREATE M:N TABLES FROM MAIN DATASET

In [5]:
dataset_path = "../../dataset/cleaned/dataset_cleaned.csv"
df_dataset = pd.read_csv(dataset_path)
df_dataset['id'] = range(1, len(df_dataset) + 1)

In [6]:
#AUTHOR MN TABLE
authors_mn_table = create_mn_table(df_dataset, id_column = 'id', key_column = 'authors')

#add unknown id instead of missing id in mn table
unknown_author_id = df_authors.loc[df_authors['author_name'] == 'Unknown', 'author_id'].iloc[0]
authors_mn_table['authors_id'] = authors_mn_table['authors_id'].replace('', unknown_author_id)

authors_mn_table['authors_id'] = authors_mn_table['authors_id'].str.strip()

authors_mn_table['authors_id'] = (authors_mn_table['authors_id']
    .map(df_authors_map).fillna(authors_mn_table['authors_id'])
    .infer_objects(copy=False).astype('Int64'))

In [ ]:

#CATHEGORIES MN TABLE
categories_mn_table = create_mn_table(df_dataset, id_column = 'id', key_column = 'categories')

print(categories_mn_table.loc[categories_mn_table['categories_id'] == ''])


categories_mn_table['categories_id'] = (categories_mn_table['categories_id']
    .replace('', np.nan)  
    .map(df_categories_map)
    .fillna(categories_mn_table['categories_id']))  

              id  main_id categories_id
1390099  1390100   328665              
2459394  2459395   636324              
2888189  2888190   774007              
3893958  3893959  1054359              
4040637  4040638  1103243              
4050848  4050849  1106382              


SAVE FILES

In [22]:

def save_dataframes(df_dict, folder_path, file_format='csv'):
    if not os.path.exists(folder_path) or not os.path.isdir(folder_path):
        raise FileNotFoundError(f"Dictionary '{folder_path}' does not exist.")

    for file_name, df in df_dict.items():
        file_name = f"{file_name}.{file_format}" 
        file_path = os.path.join(folder_path, file_name)  

        if file_format == 'csv':
            df.to_csv(file_path, index=False, encoding='utf-8')
        elif file_format == 'xlsx':
            df.to_excel(file_path, index=False, engine='openpyxl')
        else:
            raise ValueError("Unsuported file format.")

        print(f"✅ File was save: {file_path}")



In [23]:
dfs = {'final_dataset': df_dataset, 'final_categories': df_categories,
       'final_authors': df_authors,'mn_author': authors_mn_table,
       'mn_categories': categories_mn_table}
folder = "../../dataset/to_sql"

save_dataframes(dfs, folder)


✅ File was save: ../../dataset/to_sql\final_dataset.csv
✅ File was save: ../../dataset/to_sql\final_categories.csv
✅ File was save: ../../dataset/to_sql\final_authors.csv
✅ File was save: ../../dataset/to_sql\mn_author.csv
✅ File was save: ../../dataset/to_sql\mn_categories.csv


In [24]:
#formats.csv was not transfomed
file_name = "formats_cleaned.csv"
final_name = "final_formats.csv"
folder_path = "../../dataset/cleaned"
final_path = "../../dataset/to_sql"

source_path = os.path.join(folder_path, file_name)
destination_path = os.path.join(final_path, final_name)

with open(source_path, "rb") as src, open(destination_path, "wb") as dst:
    dst.write(src.read())

print(f"✅ File was save")

✅ File was save
